# Python -> LLVM compiler

In [2]:
%run ~/.jupyter/config.ipy

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import ast_to_llvm.python_to_core as ptc
import ast_to_llvm.core_to_llvm as ctl

# tests

In [4]:
func = """
def f(a, b):
    c = a + b
    return c
"""

In [7]:
core = ptc.PythonVisitor()(func)[0]


def f(a, b):
    c = a + b
    return c

CORE
('Func',
 {'args': [('Var', {'ref': "'a'", 'type': None}),
           ('Var', {'ref': "'b'", 'type': None})],
  'body': [('Assign',
            {'ref': "'c'",
             'type': None,
             'val': ('PrimOp',
                     {'args': [('Var', {'ref': "'a'", 'type': None}),
                               ('Var', {'ref': "'b'", 'type': None})],
                      'fn': "'add#'"})}),
           ('Return', {'val': "'c'"})],
  'fname': "'f'"})


In [12]:
ctl.LLVMEmitter().visit(core)

Visit Func
('Func',
 {'args': [('Var', {'ref': "'a'", 'type': None}),
           ('Var', {'ref': "'b'", 'type': None})],
  'body': [('Assign',
            {'ref': "'c'",
             'type': None,
             'val': ('PrimOp',
                     {'args': [('Var', {'ref': "'a'", 'type': None}),
                               ('Var', {'ref': "'b'", 'type': None})],
                      'fn': "'add#'"})}),
           ('Return', {'val': "'c'"})],
  'fname': "'f'"})
Visit Assign
Visit PrimOp
Visit Return
